In [162]:
#HW4

not.installed <- function(pkg) !is.element(pkg, installed.packages()[,1])

if (not.installed("ggplot2")) install.packages("ggplot2")
if (not.installed("devtools")) install.packages("devtools")
if (not.installed("DAAG")) install.packages("DAAG")
if (not.installed("InformationValue")) install.packages("InformationValue") 

library(ggplot2)
library(MASS)
library(DAAG)  
library(lattice)
library(InformationValue)  
    
#library(devtools)
#install_github("ggbiplot", "vqv")
    
  

In [163]:
diamonds = subset( diamonds, (x>0) & (y>0) & (z>0) )  #  There are actually some zero values, we omit them.
numeric.diamonds = transform( diamonds,
                              cut = as.numeric(unclass(diamonds$cut)),
                              color = as.numeric(unclass(diamonds$color)),
                              clarity = as.numeric(unclass(diamonds$clarity))
                   ) 

In [164]:
MY_UID = 004870721 ########## you must enter your UCLA UID here !!!

set.seed( MY_UID )

n = nrow( numeric.diamonds )
sample.size = 0.75 * n   ###### Use 75% of the data for the training set
training.row.ids = sample( (1:n), sample.size )

my.training.set = numeric.diamonds[  training.row.ids, ]
my.test.set     = numeric.diamonds[ -training.row.ids, ]   # set complement of training.set.ids

head(numeric.diamonds)

carat,cut,color,clarity,depth,table,price,x,y,z
0.23,5,2,2,61.5,55,326,3.95,3.98,2.43
0.21,4,2,3,59.8,61,326,3.89,3.84,2.31
0.23,2,2,5,56.9,65,327,4.05,4.07,2.31
0.29,4,6,4,62.4,58,334,4.20,4.23,2.63
0.31,2,7,2,63.3,58,335,4.34,4.35,2.75
0.24,3,7,6,62.8,57,336,3.94,3.96,2.48


In [165]:
classification_accuracy = function( model, test.data, test.solutions ) {
    predictions = predict(model, test.data)
    incorrect.predictions  =  (predictions$class != test.solutions)
    incorrect.ids = test.solutions[incorrect.predictions]
    confusion.matrix = table( test.solutions, predictions$class )
    
    accuracy = (length(test.solutions) - length(incorrect.ids)) / length(test.solutions)
    return (accuracy)

}

linear_regression_accuracy = function( model, test.data, test.solutions ) {
       
    predictions = predict(model, test.data)    
    actuals_preds = data.frame(cbind(actuals=test.solutions, predicteds=predictions))  # make actuals_predicteds dataframe.  
    correlation_accuracy = cor(actuals_preds)  # 82.7%   
    min_max_accuracy = mean(apply(actuals_preds, 1, min) / apply(actuals_preds, 1, max))    
    mape = mean(abs((actuals_preds$predicteds - actuals_preds$actuals))/actuals_preds$actuals)          
    
#     #k-validation graphs
#     cvResults = suppressWarnings(cv.lm(numeric.diamonds, model, m=5, dots=FALSE, seed=MY_UID, legend.pos="topleft",  printit=FALSE, main="K-folds Validation")) 
 
#     cat('AIC (lower the better): ', AIC(model), '\n')
#     cat('BIC (lower the better): ', BIC(model), '\n')
    
#     cat('min_max_accuracy (higher the better): ', min_max_accuracy, '\n')  
#     cat('mape (lower the better): ', mape, '\n')   
#     print(summary(model))
    
#     cat('mean squared error (lower the better): ',attr(cvResults, 'ms'), '\n')
   
    rsquared = summary(model)$r.squared
    return(rsquared)
}

logistic_regression_accuracy = function( model, test.data, test.solutions ) {
    #because glm returns 0 or 1, we can compare it against test data to calculate accuracy.
    
    Predictions = predict( model, newdata = test.data, type = "response")
    PredictionClasses = round(Predictions) # Turn the numeric values into {0,1} values
    
    PredictionsIncorrect = (PredictionClasses != test.solutions)
    
    sum(PredictionsIncorrect) # number of incorrect predictions

    
    (ConfusionMatrix = table( PredictionClasses, test.solutions ) )
    
    #print(ConfusionMatrix)
    
    accuracy = sum(diag(ConfusionMatrix)) / nrow(test.data)
    
    #accuracy = (length(test.solutions) - length(incorrect.ids)) / length(test.solutions)
    
    return (accuracy)
    
}



In [172]:
#base line models
baseline_m1 = qda( cut ~ .,           data=my.training.set )
baseline_m1_accuracy = classification_accuracy(baseline_m1, my.test.set, my.test.set$cut)

baseline_m2 = lm(  price ~ .,         data=my.training.set )
baseline_m2_accuracy = linear_regression_accuracy(baseline_m2, my.test.set, my.test.set$price)

baseline_m3 = lm(  log10(price) ~ .,  data=my.training.set )
baseline_m3_accuracy = linear_regression_accuracy(baseline_m3, my.test.set, my.test.set$price)

baseline_m4 = suppressWarnings( glm( I(price>1500) ~ ., data=my.training.set, family=binomial ))
baseline_m4_accuracy = logistic_regression_accuracy( baseline_m4, my.test.set, I(my.test.set$price>1500) ) 


In [173]:
#improved models
m1 = qda(cut ~ price + carat + depth + table, data=my.training.set)
m1_accuracy = classification_accuracy(m1, my.test.set, my.test.set$cut)

m2 = lm(price ~ (carat + cut + color + clarity + depth + table)^2 + x + y + z, data=my.training.set )
m2_accuracy = linear_regression_accuracy(m2, my.test.set, my.test.set$price)

m3 = lm(log10(price) ~ (carat + cut + color + clarity + depth + table)^2 + log10(x) + log10(y) + log10(z), data=my.training.set)
m3_accuracy = linear_regression_accuracy(m3, my.test.set, my.test.set$price)

m4 = suppressWarnings( glm( I(price>1500) ~ (carat + cut + color + clarity + depth + table)^2, data=my.training.set, family=binomial ))
m4_accuracy = logistic_regression_accuracy( m4, my.test.set, I(my.test.set$price>1500) ) 



In [174]:
cat(baseline_m1_accuracy,', qda( cut ~ .,           data=my.training.set )', '\n')
cat(baseline_m2_accuracy,', lm(  price ~ .,         data=my.training.set )', '\n')
cat(baseline_m3_accuracy,', lm(  log10(price) ~ .,  data=my.training.set )', '\n')
cat(baseline_m4_accuracy,', glm( I(price>1500) ~ ., data=my.training.set, family=binomial )', '\n')

cat(m1_accuracy,', qda( cut ~ price + table + color + clarity, data=my.training.set )', '\n')
cat(m2_accuracy,', lm( price ~ (carat + cut + color + clarity + depth + table)^2 + x + y + z, data=my.training.set )', '\n')
cat(m3_accuracy,', lm(log10(price) ~ (carat + cut + color + clarity + depth + table)^2 + log10(x) + log10(y) + log10(z), data=my.training.set)', '\n')
cat(m4_accuracy,', glm( I(price>1500) ~ (carat + cut + color + clarity + depth + table)^2, data=my.training.set, family=binomial )', '\n')




0.5397626 , qda( cut ~ .,           data=my.training.set ) 
0.9060584 , lm(  price ~ .,         data=my.training.set ) 
0.9776148 , lm(  log10(price) ~ .,  data=my.training.set ) 
0.9784125 , glm( I(price>1500) ~ ., data=my.training.set, family=binomial ) 
0.6301187 , qda( cut ~ price + table + color + clarity, data=my.training.set ) 
0.9587075 , lm( price ~ (carat + cut + color + clarity + depth + table)^2 + x + y + z, data=my.training.set ) 
0.981834 , lm(log10(price) ~ (carat + cut + color + clarity + depth + table)^2 + log10(x) + log10(y) + log10(z), data=my.training.set) 
0.9847923 , glm( I(price>1500) ~ (carat + cut + color + clarity + depth + table)^2, data=my.training.set, family=binomial ) 
